## Frozen convolution layer for pattern manufacturing. 

# frozen_diff_conv: Creates "derivatives" for time series features. 

When given a time series like feature, e.g. [1,2,3,1,2,3,1,2,3], we would like to create the "first derivative", "second derivative", and so on by applying the frozen convolution of (-1,1) with no stride. Instead of pre-padding, we will opt for post padding. As an example, the "first derivative" of the given example time series would be [1,1,-2,1,1,-2,1,1,0] where the last zero is what I mean by post padding. 

In [31]:
import torch
import torch.nn as nn 
import torch.nn.functional as F

As a reminder, Conv1d requires input of shape (N,Channel,Length). 

In [68]:
#Created 07/01/25. Copied to training.py. 
class frozen_diff_conv(nn.Module):
    def __init__(self,n_diff=1):
        """
        A frozen 1d convolution layer that creates "n th derivative" features. It expects input of tensor shape (N,Channel,Length) with N be any arbitrary positive integer, Channel == 1, and Length be any arbitrary integer. 
        
        :param n_diff: Defaulted to 1. The number of derivative wanted. 
        :return: A tensor of shape (N, n_diff, Length). Where the n th (start from zero) tensor in the dimension 1 (we start with dimension 0) is the n th "derivative" of the imput tensor. However, if n_diff >= Length, None will be returned. 
        """
        super().__init__()
        self.n_diff=n_diff
        self.frozen_conv=nn.Conv1d(1,1,kernel_size=2,bias=False)
        with torch.no_grad():
            self.frozen_conv.weight[:]=torch.tensor([[[-1.0,1.0]]])
            # self.frozen_conv.bias.zero_()

    def forward(self,x):
        out_tensor=x
        x_diff=x
        #Check if the user is taking too many derivatives 
        if self.n_diff>=x.shape[2]: 
            print("Too many derivatives. Returning None.\n")
            return None
        for diff in range(1,self.n_diff+1): 
            x_diff=self.frozen_conv(x_diff)
            x_diff_pad=F.pad(x_diff,(0,diff),mode="constant",value=0)
            out_tensor=torch.cat((out_tensor,x_diff_pad),dim=1)
            
        # x_diff=self.frozen_conv(x)
        # x_diff_pad=F.pad(x_diff,(0,1),mode="constant",value=0)
        # out_tensor=torch.cat((x,x_diff_pad),dim=1)
        
        return out_tensor
    

# An example of frozen_diff_conv. 

In [69]:
import importlib
import sys

sys.path.append("../")

from proj_mod import training
importlib.reload(training);

In [73]:
frozen_conv=training.frozen_diff_conv(n_diff=2)

In [74]:
x=torch.tensor([[[1.0,2.0,3.0,1.0,2.0,3.0,1.0,2.0,3.0]]])

In [75]:
frozen_conv(x)

tensor([[[ 1.,  2.,  3.,  1.,  2.,  3.,  1.,  2.,  3.],
         [ 1.,  1., -2.,  1.,  1., -2.,  1.,  1.,  0.],
         [ 0., -3.,  3.,  0., -3.,  3.,  0.,  0.,  0.]]],
       grad_fn=<CatBackward0>)